# Test Door Key Offline Training with d3rlpy and Decision Transformer. Comparing with BC and CQL

We will make new experiments in Door Key 16x16 environment from Minigrid Gym with Offline algorithms from d3rlpy. 

We test the following offline algorithms:
* Discrete Decision Transformer (DT)
* Discrete Behaviour Clonig (BC)
* Discrete CQL (CQL)

## References
We will use the docs of d3rlpy to describe the algorithms. 
https://d3rlpy.readthedocs.io/en/v2.3.0/references/algos.html

In [ ]:
# Test if we are running on CoLab or not
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  !apt-get install -y xvfb ffmpeg > /dev/null 2>&1
  %pip install pyvirtualdisplay pygame moviepy > /dev/null 2>&1
  %pip install d3rlpy
  %pip install matplotlib
else:
  print('Not running on CoLab')

In [ ]:

import os
path = "./models"
path = "./datasets"
path = "./videos/video-doorkey-dt-d3rlpy"

In [ ]:
import gymnasium as gym
from gymnasium import spaces
from minigrid.envs import DoorKeyEnv
from gymnasium.core import ActType, ObsType
from typing import Any, SupportsFloat
import random, math
import numpy as np

from gymnasium.envs.registration import register

register(
    id="MiniGrid-DoorKey-16x16-v0",
    entry_point="minigrid.envs:DoorKeyEnv",
    kwargs={"size": 16},
)
    

We created a PartialObsWrapper to augment flattened partial observations with the agent's direction in a single vector. This is beneficial for training the Decision Transformer algorithm.

Additionally, we developed a PartialAndFullyObsWrapper to enhance flattened partial observations with the agent's direction in just one vector, and to append the full observation to the partial observation. This is advantageous for sampling data from the environment using the trained DQN agent, and for saving the data in a dataset.

In [ ]:
import minigrid
from minigrid.core.constants import COLOR_TO_IDX, OBJECT_TO_IDX
from gymnasium.core import ObservationWrapper

class PartialObsWrapper(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)

        new_image_space = spaces.Box(
            low=0,
            high=255,
            shape=(((self.env.agent_view_size * self.env.agent_view_size *  3) + 1),),  
            dtype="uint8",
        )

        self.observation_space = spaces.Dict(
            {**self.observation_space.spaces, "image": new_image_space}
        )

    def observation(self, obs):
        image = np.concatenate((obs["image"].flatten(), np.array([obs["direction"]])))

        return {"image": image}

class PartialAndFullyObsWrapper(ObservationWrapper):

    def __init__(self, env):
        super().__init__(env)

        new_image_space = spaces.Box(
            low=0,
            high=255,
            shape=(self.env.width, self.env.height, 3),  # number of cells
            dtype="uint8",
        )

        self.observation_space = spaces.Dict(
            {**self.observation_space.spaces, "image": new_image_space}
        )

    def observation(self, obs):
        env = self.unwrapped
        full_grid = env.grid.encode()
        full_grid[env.agent_pos[0]][env.agent_pos[1]] = np.array(
            [OBJECT_TO_IDX["agent"], COLOR_TO_IDX["red"], env.agent_dir]
        )

        partial_image = np.concatenate((obs["image"].flatten(), np.array([obs["direction"]])))

        return {"partial_image": partial_image, "image": full_grid}

def create_env(env_key, max_episode_steps=100, isPartialObs=False, isFullyObs=True, is_video=False):
    
    render_mode = None

    if is_video == True:
        render_mode = 'rgb_array'

    env = gym.make(env_key, max_episode_steps=max_episode_steps, render_mode=render_mode, see_through_walls=True)

    if isPartialObs and isFullyObs:
        env =  PartialAndFullyObsWrapper(env)
    elif isPartialObs:
        env = PartialObsWrapper(env)
        env = minigrid.wrappers.ImgObsWrapper(env)
    elif isFullyObs:
        env = minigrid.wrappers.FullyObsWrapper(env)
        env = minigrid.wrappers.ImgObsWrapper(env)

    return env


In [ ]:
import d3rlpy

config = {
    "seed": 1,
    "dataset_size": 10000,
    "epsilon": 0.3,
    "max_episode_steps": 200,
    "experiment_name": "door-key-16x16",
    "device": "cuda:0"
}

env_key = "MiniGrid-DoorKey-16x16-v0"


We develop a CustomReplayBuffer to exclude episodes that have zero or negative rewards. Our goal is to retain only those episodes in which the agent is making progress towards the goal.

In [ ]:
import numpy as np
from tqdm import tqdm


class CustomReplayBuffer(d3rlpy.dataset.ReplayBuffer):
    ' Custom Replay Buffer to clip episodes with negative rewards'
    
    def clip_episode(self, terminated: bool) -> None:
        r"""Clips the current episode.

        Args:
            terminated: Flag to represent environment termination.
        """

        episode_to_remove = None
        # Check if the episode's reward is 0 or negative
        if not terminated and self._writer._active_episode.rewards.mean() <= 0:
            episode_to_remove = self._writer._active_episode
            
        self._writer.clip_episode(terminated)

        if episode_to_remove is not None:
            # Remove all transitions associated with the episode to remove
            self._buffer._transitions = [(ep, idx) for ep, idx in self._buffer._transitions if ep is not episode_to_remove]
            self._buffer.episodes.remove(episode_to_remove)  



In [ ]:
env = create_env(env_key, max_episode_steps=config["max_episode_steps"], isPartialObs=True, isFullyObs=False)
env_eval = create_env(env_key, max_episode_steps=config["max_episode_steps"], isPartialObs=True, isFullyObs=False)

buffer = d3rlpy.dataset.InfiniteBuffer()
dataset = CustomReplayBuffer(buffer, env=env, cache_size=config["max_episode_steps"])

dataset.load('./datasets/dataset_door-key-dqn-16x16-mixed.d3', buffer=buffer)

experiment_name = config["experiment_name"]
experiment_seed = config["seed"]

## Decision Transformer
Decision Transformer solves decision-making problems as a sequence modeling problem. It is a Transformer-based architecture that learns to predict the next action given the current state and the history of previous actions and states. 

In [ ]:
dt = d3rlpy.algos.DiscreteDecisionTransformerConfig(
    batch_size=64,
    encoder_factory=d3rlpy.models.VectorEncoderFactory(
    ),
    optim_factory=d3rlpy.models.GPTAdamWFactory(),
    position_encoding_type=d3rlpy.PositionEncodingType.GLOBAL,
    context_size=60,
    num_heads=8,
    num_layers=3,
    max_timestep=200,
).create(device=config["device"])

dt.fit(
    dataset,
    n_steps=5000,
    n_steps_per_epoch=1000,
    experiment_name=f"DT_{experiment_name}_{experiment_seed}"
)

## Behaviour Cloning (BC)

Behavior Cloning (BC) is to imitate actions in the dataset via a supervised learning approach. Since BC is only imitating action distributions, the performance will be close to the mean of the dataset even though BC mostly works better than online RL algorithms.

In [ ]:
bc = d3rlpy.algos.DiscreteBCConfig().create(device="cuda:0")

bc.fit(
   dataset,
   n_steps=10000,
   n_steps_per_epoch=1000,
   experiment_name=f"BC_{experiment_name}_{experiment_seed}"
)



## Conservative Q-Learning algorithm (CQL)

Config of Conservative Q-Learning algorithm.

CQL is a SAC-based data-driven deep reinforcement learning algorithm, which achieves state-of-the-art performance in offline RL problems.

CQL mitigates overestimation error by minimizing action-values under the current policy and maximizing values under data distribution for underestimation issue.

In [ ]:
cql = d3rlpy.algos.DiscreteCQLConfig().create(device="cuda:0")

cql.fit(
   dataset,
   n_steps=10000,
   n_steps_per_epoch=1000,
   experiment_name=f"CQL_{experiment_name}_{experiment_seed}"
)

We train the three algorithms with the same dataset, and we compare the results.

In [ ]:
# We are going to compare the performance of DQN and DT

trials = 1000

def evaluate_policy(env, actor, explorer=None, n_trials=10):

    success = 0
    for _ in tqdm(range(n_trials)):
        
        if (explorer is  None):
            actor.reset()
        obs,_ = env.reset()

        done, truncated = False, False
        while not (done or truncated):
            if explorer is not None:
                x = np.expand_dims(obs, axis=0)
                action = explorer.sample(actor, x, 0)[0]
            else:
                action = actor.predict(obs, 1)
            obs, reward, done, truncated, _ = env.step(action)
            if done and reward > 0:
                success += 1
    return success / n_trials

env = create_env(env_key, max_episode_steps=config["max_episode_steps"], isPartialObs=True, isFullyObs=False)

actor_dt = dt.as_stateful_wrapper(
    target_return=1,
    action_sampler=d3rlpy.algos.SoftmaxTransformerActionSampler(temperature=1.0,)
)
dt_score = evaluate_policy(env, actor_dt, n_trials=trials)

explorer = d3rlpy.algos.ConstantEpsilonGreedy(epsilon=config["epsilon"])

bc_score = evaluate_policy(env, bc, explorer, n_trials=trials)
cql_score = evaluate_policy(env, cql, explorer, n_trials=trials)

print("BC score:", bc_score)
print("CQL score:", cql_score)
print("DT score:", dt_score)



In [ ]:
# We will make a bar chart to compare the performance of DQN and DT

import matplotlib.pyplot as plt

labels = ['DT', 'BC', 'CQL']
scores = [dt_score, bc_score, cql_score]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(8, 6))
rects1 = ax.bar(x - width/2, scores, width, label='Scores')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by offline algorithms DT, BC and CQL')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()

plt.show()

